# Question Answer
- Index
    - Create an index
    - Query the index directly with LLM
    - Query the index via Chain
- Vector Store
    - Create a vector store
    - Query the vector store directly with LLM
    - Query the vector store via Chain

---

## Setup

In [ ]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_type = os.environ.get("OPENAI_API_TYPE")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = os.environ.get("OPENAI_API_VERSION")

In [ ]:
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name="gpt4",
    temperature=0,
)

## Index

### Create an index
We create index when the documents are not small and we need to break them into smaller chunks. This is done to improve the performance of the search.

In [ ]:
from langchain.document_loaders import CSVLoader

file = ".../data/OutdoorClothingCatalog_1000_small.csv"
loader = CSVLoader(file_path=file)
docs = loader.load()

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import HuggingFaceEmbeddings

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch, embedding=HuggingFaceEmbeddings()
).from_loaders([loader])

### Query the index directly with LLM
Fetches all the relevant documents from the index and then sends them as context to the LLM model.

In [ ]:
from IPython.display import display, Markdown

query = "Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

response = index.query(query, llm)
display(Markdown(response))

### Query the index via Chain
Fetches all the relevant documents from the index and then sends them as context to the LLM model.

In [ ]:
from langchain.chains import RetrievalQA

qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
)

response = qa_stuff.run(query)
display(Markdown(response))

## Vector Store

### Create a vector store directly from the documents
When the documents are small, we don't need indexing. We can directly create a vector store from the documents.

In [ ]:
db = DocArrayInMemorySearch.from_documents(
    documents=docs, embedding=HuggingFaceEmbeddings()
)

### Query the vector store directly with LLM
Fetch all the relevant documents from the vector store and then send them as context to the LLM model.

In [ ]:
# embed = embeddings.embed_query(query)
docs = db.similarity_search(query)
docs

In [ ]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])
response = llm.call_as_llm(f"{qdocs} Question: {query}")
display(Markdown(response))

### Query the vector store via Chain
Fetches all the relevant documents from the vector store and then sends them as context to the LLM model.

In [ ]:
from langchain.chains import RetrievalQA

qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=db.as_retriever(), verbose=True
)

response = qa_stuff.run(query)
display(Markdown(response))